In [ ]:
# often used imports
import os
import re
import sys
from time import sleep
from pathlib import Path
from pickle import dump
import numpy as np
import scipy as sp
import pandas as pd
from tqdm.auto import tqdm
from operator import itemgetter
from sklearn.impute import KNNImputer
import json

# turn off warnings and set up logging
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

# environment variables
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style('whitegrid')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)

# automatically reload packages when they are edited
%load_ext autoreload
%autoreload 2

from blood_response.gwas_preprocessing import *

### configure paths and names

In [ ]:
analysis_name = 'sysmex_custom_gates_v9'
gwas_run = '2020_10_28'
gwas_path = f'/mnt/obi0/phi/gwas/obi_gwas/runs/{gwas_run}/imputed_plus_biobank/'
analysis_path = f'/mnt/obi0/phi/gwas/gwas_analyses/{analysis_name}-obi{gwas_run}/'
patient_ids = pd.read_csv(f'{gwas_path}/patient_ids_all.tsv', sep='\t', dtype=str)
covariates_df = pd.read_parquet('/mnt/obi0/phi/ehr/obi_biobank_annotations/covariates_21-07-06.parquet')

perturbation_names = pd.read_excel(analysis_path + '/perturbation.time.dose.ID.summary.xlsx')
perturbation_names['id_str'] = perturbation_names.id_number.apply(lambda x: f'{x:03d}')

In [ ]:
# redcap with blood draw times
redcap_blood_draws = pd.read_parquet('/mnt/obi0/phi/sysmex/redcap/2021_07_06/redcap_phlebotomy_validated.parquet')
redcap_blood_draws.SpecimenTakenTimeDTS.dt.hour.hist(bins=9)

# medications, transplants etc
anticoagulants_30d = \
pd.read_parquet('/mnt/obi0/mhomilius/projects/sysmex/data/obi_biobank_annotations/anticoagulant_obi_30d_21-07-06.parquet')
steroids_30d = \
pd.read_parquet('/mnt/obi0/mhomilius/projects/sysmex/data/obi_biobank_annotations/steroid_obi_30d_21-07-06.parquet')
steroids_30d = steroids_30d[['PatientID', 'Glucocorticoids']]
steroids_30d['glucocorticoid'] = steroids_30d.Glucocorticoids == 1.0
covariates_df = covariates_df.merge(steroids_30d[['PatientID', 'glucocorticoid']], how='left')

transplants = \
pd.read_parquet('/mnt/obi0/mhomilius/projects/sysmex/data/obi_biobank_annotations/transplant_with_lvad_21-07-06.parquet')
covariates_df = covariates_df.merge(transplants, how='left')

# use this subset of covariates
covariates_df = \
covariates_df[['PatientID', 'rc_consent_age', 'SexDSC', 'Race',
               'BMI', 'TobaccoUserDSC', 'glucocorticoid', 'transplant']].\
rename(columns={
    'rc_consent_age': 'Age',
    'SexDSC': 'Sex',
    'BMI': 'BMI',
    'TobaccoUserDSC': 'Tobacco',
    'glucocorticoid': 'Glucocorticoid',
    'transplant': 'Transplant'
})

# this is based on crawling the fcs files on the server
sysmex_fcs_meta_all = pd.read_parquet('/mnt/obi0/mhomilius/projects/sysmex/data/binned_sysmex/obi_20-09-01/meta_matched.parquet')
sysmex_fcs_meta_all = sysmex_fcs_meta_all.drop_duplicates(subset="FIL")

sysmex_fcs_meta = sysmex_fcs_meta_all.\
reset_index().\
rename(columns={'index': 'sample_id'})[['sample_id', 'FIL', 'DATE', 'ETIM']]
# 37 samples were run more than once, but de-duplicating those after merge based on FIL
(sysmex_fcs_meta.value_counts('sample_id') > 1).sum()

### basic input data cleanup (sample ids, perturbation ids, errors)

In [ ]:
#for channel in ['wdf', 'ret', 'wnr', 'plt-f']:
channel = 'wdf'

print('creating directories and filtering samples')

Path(f'{analysis_path}/gwas/{channel}/perturbations/').mkdir(parents=True, exist_ok=True)
gated_stats = f'{analysis_path}/{channel.upper()}_complete.csv'
sysmex_results = pd.read_csv(gated_stats)

gated_stats2 = f'{analysis_path}/{channel.upper()}_short.csv'
sysmex_results2 = pd.read_csv(gated_stats2).rename(columns={'Sample': 'FIL'})
sysmex_results2 = sysmex_results2[['FIL'] + [c for c in sysmex_results2.columns if c.endswith('Percentage')]]

# sysmex results contains gated stats after processing the FCS files with FlowJo
# but otherwise only minimal filtering
sysmex_results = sysmex_results.rename(columns={'Unnamed: 0': 'FIL'}).drop('Unnamed: 111', axis=1)

# also drop the mean and sd from FlowJo
sysmex_results = sysmex_results.loc[~sysmex_results.FIL.isin(['Mean', 'SD'])]

# there should not be any FIL duplicates, but de-duplicating just in case
sysmex_results = sysmex_results.drop_duplicates(subset='FIL')
sysmex_results = sysmex_results.merge(sysmex_results2, left_on='FIL', right_on='FIL', how='left')
sysmex_results = sysmex_results.merge(sysmex_fcs_meta, left_on='FIL', right_on='FIL', how='left')
sysmex_results = sysmex_results.loc[~sysmex_results.sample_id.isna()]
sysmex_results = sysmex_results.drop_duplicates('sample_id')
sysmex_results['instrument'] = sysmex_results['FIL'].apply(lambda x:x.split("^")[0][1:])

# Indicate error files
sysmex_results['non_error'] = sysmex_results['sample_id'].str.contains(".*-.*-.*",regex= True)
sysmex_results['patient_sample_id'] = sysmex_results['sample_id'].apply(lambda x:x.rsplit("-",1)[0] if any(pd.Series(x).str.contains(".*-.*-.*",regex= True)) else "error")
sysmex_results['perturbation_num'] = sysmex_results['sample_id'].apply(lambda x:x.rsplit("-",1)[1] if any(pd.Series(x).str.contains(".*-.*-.*",regex= True)) else "error")

# this is the distribution of perturbations/error files
print(sysmex_results.shape)
print(sysmex_results['perturbation_num'].value_counts())

# Remove any XN-450 samples
sysmex_results = sysmex_results.loc[~(sysmex_results.instrument == 'XN-450')]

# Remove samples from other studies
sysmex_results = sysmex_results[~sysmex_results["patient_sample_id"].str.contains("PET")]
sysmex_results = sysmex_results[~sysmex_results["patient_sample_id"].str.contains("FH")]
sysmex_results = sysmex_results[~sysmex_results["patient_sample_id"].str.contains("AZ")]

# drop any perturbation that is not in [0-36], except 20 (water on the 450)
sysmex_results = sysmex_results.loc[sysmex_results.perturbation_num.isin(
    [f'{i:03d}' for i in range(40) if i != 20]
)]
print(sysmex_results.shape)
print(sysmex_results['perturbation_num'].value_counts())

# standardize the ids so they match
sysmex_results['standard_study_id'] = sysmex_results['patient_sample_id'].apply(standardize_study_id)
sysmex_results = sysmex_results.merge(patient_ids[['standard_study_id', 'PatientID']],
                     left_on='standard_study_id',
                     right_on='standard_study_id',
                     how='left')

# removing subjects not in redcap
print(f'subjects without patientid: {sysmex_results.loc[sysmex_results.PatientID.isna()].shape}')
sysmex_results = sysmex_results.loc[~sysmex_results.PatientID.isna()]

# simplify feature names and shorten phenotype names
sysmex_results.columns = [re.sub('[^0-9a-zA-Z]+', '_', c) for c in sysmex_results.columns]
sysmex_results.columns = sysmex_results.columns.str.\
    replace('Signal_', '').str.\
    replace('Side_Fluorescence_', 'SFL').str.\
    replace('Side_Scatter_', 'SSC').str.\
    replace('Forward_Scatter_', 'FSC').str.\
    replace('Robust_CV', 'CV').str.\
    replace('Robust_SD', 'SD').str.\
    replace('Median_', 'Med_')

# add the NE2/NE4 ratio trait
sysmex_results['NE2_NE4_ratio'] = sysmex_results.NE2_Count / sysmex_results.NE4_Count

pheno_cols = list(set(sysmex_results.columns).difference(
    [
        'FIL', 'DATE', 'ETIM',
        'sample_id', 'patient_sample_id', 'standard_study_id',
        'PatientID', 'instrument', 'non_error',
        'perturbation', 'perturbation_num'
    ]
))

# merge with covariates and sysmex metadata like date and time of measurement
sysmex_results = sysmex_results.merge(covariates_df, left_on='PatientID', right_on='PatientID')

# drop measurments after May 2020
sysmex_results['sysmex_datetime'] = pd.to_datetime(sysmex_results.DATE + ' ' + sysmex_results.ETIM, dayfirst=True)
sysmex_results['DATE'] = pd.to_datetime(sysmex_results.DATE)
sysmex_results = sysmex_results.loc[sysmex_results.DATE < pd.Timestamp("2020-05-01")]

# use distinct perturbation ids for 006, 007, 008 before 2019-08-14
sysmex_results.loc[
    (sysmex_results.perturbation_num == '006') &
    (sysmex_results.sysmex_datetime < pd.to_datetime('2019-08-14')), 'perturbation_num'] = '037'

sysmex_results.loc[
    (sysmex_results.perturbation_num == '007') &
    (sysmex_results.sysmex_datetime < pd.to_datetime('2019-08-14')), 'perturbation_num'] = '038'

sysmex_results.loc[
    (sysmex_results.perturbation_num == '008') &
    (sysmex_results.sysmex_datetime < pd.to_datetime('2019-08-14')), 'perturbation_num'] = '039'

# add perturbation names
sysmex_results = sysmex_results.merge(
    perturbation_names[['id_str', 'figure_name']].rename(
        columns={'figure_name':'Perturbation', 'id_str': 'perturbation_num'}),
    left_on='perturbation_num',
    right_on='perturbation_num'
)

# save the cleaned data to disk (but outliers have not been removed yet)
sysmex_results.to_parquet(f'{analysis_path}/{channel}_cleaned.parquet')
sysmex_results.sysmex_datetime.dt.hour.hist(bins=14)

### outlier removal and plotting to see batch effects

In [ ]:
sysmex_results = sysmex_results.merge(
    redcap_blood_draws[['standard_study_id', 'SpecimenTakenTimeDTS', 'rc_consent_datetime' ]]
)

sysmex_results['draw_analysis_timedelta'] = sysmex_results.sysmex_datetime - sysmex_results.SpecimenTakenTimeDTS
sysmex_results['draw_analysis_hours'] = sysmex_results.draw_analysis_timedelta / np.timedelta64(1, 'h')

# these are suspicious/incorrect time stamps
sysmex_results.loc[(sysmex_results.draw_analysis_hours < 0) |  (sysmex_results.draw_analysis_hours > 38)].\
to_excel(f'{analysis_path}/time_mismatch.xlsx')

# drop the incorrect times
sysmex_results = \
sysmex_results.\
loc[~((sysmex_results.draw_analysis_hours < 0) |  (sysmex_results.draw_analysis_hours > 38))]
sysmex_results.draw_analysis_hours.hist()

In [ ]:
filtered_sysmex_results = []
for perturb_ in tqdm(sorted(list(sysmex_results.Perturbation.unique()))):
    perturb_results = sysmex_results.copy(deep=True).loc[sysmex_results.Perturbation == perturb_]
    outliers = ~filter_outliers_mad(perturb_results, 'draw_analysis_hours', outlier_cutoff=2.5, return_stats=False)
    print(f'dropping blood draw outliers: {outliers.sum()} / {len(outliers)}')
    # removes about 1%-2% of samples for WDF
    filtered_sysmex_results.append(perturb_results.loc[~outliers])

In [ ]:
sysmex_results = pd.concat(filtered_sysmex_results)

In [ ]:
# make a copy of the phenotype data for imputation, outlier removal etc
pheno_df = sysmex_results.copy(deep=True)

print('KNN imputation')
# impute missing values for outlier removal
imputer = KNNImputer(n_neighbors=5, weights="uniform")
transformed = imputer.fit_transform(pheno_df[pheno_cols])
pheno_df[list(pheno_cols)] = transformed

In [ ]:
print('global outlier removal')
if not Path(analysis_path + f'/projections/{channel}/').exists():
    # project with PCA, ICA and UMAP, flag outliers and make plots with various covariates
    pheno_df_projected = project_phenotypes(
        pheno_df,
        pheno_cols,
        output_dir=analysis_path + f'/projections/{channel}/',
        impute=False, load_embeddings=False,
        make_plots=True, plot_outliers=True
    )
else:
    # project with PCA, ICA and UMAP, flag outliers and make plots with various covariates
    pheno_df_projected = project_phenotypes(
        pheno_df,
        pheno_cols,
        output_dir=analysis_path + f'/projections/{channel}/',
        impute=False, load_embeddings=True,
        make_plots=False, plot_outliers=False
    )

In [ ]:
# set to the original values to remove imputed values
pheno_df_projected[pheno_cols] = sysmex_results[pheno_cols]

print(f'dropping outliers: {pheno_df_projected.outlier.sum()}')
# drops about 1% of samples (500 out of 54k) for WDF
pheno_df_projected = pheno_df_projected.loc[~pheno_df_projected.outlier]

In [ ]:
# now save the processed data
pheno_df_projected.drop(['draw_analysis_timedelta'], axis=1).to_parquet(f'{analysis_path}/{channel}_pheno_projected.parquet')
    
# use the outlier-filtered data as GWAS input
gwas_input = pheno_df_projected

### prepare data for plink

In [ ]:
# loop over perturbations and create input data for plink
for perturb_ in tqdm(sorted(list(gwas_input.Perturbation.unique()))):
    print(f'\n\n\n#### {perturb_} ####')
    perturb_results = gwas_input.copy(deep=True).loc[gwas_input.Perturbation == perturb_]
    perturb_results = perturb_results.drop(
        ['FIL', 'sample', 'patient_sample_id', 'instrument',
         'Perturbation', 'standard_study_id',
         'DATE', 'ETIM', 'Perturbation',
         'analysis_date', 'analysis_time', 'outlier', 'non_error',
         'SpecimenTakenTimeDTS', 'rc_consent_datetime', 'draw_analysis_timedelta'
        ],
        axis=1,
    )
    
    output_folder = f'{analysis_path}/gwas/{channel}/perturbations/{perturb_}/'
    if not Path(output_folder).exists():
        Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # make a full plot of all data including quantile transformed data to understand distributions
    pheno_data_all = perturb_results.copy(deep=True)
    quantile_data = pheno_quantile_transform(pheno_data_all[pheno_cols], pheno_cols)
    pheno_data_all = pheno_data_all.merge(quantile_data, left_index=True, right_index=True)
    pheno_columns_all = sorted(pheno_cols + list(quantile_data.columns))
    plot_pheno(pheno_data_all, pheno_columns_all, output_folder + 'pheno_all.pdf')
    
    # now remove outliers for each individual trait and condition
    # first do quantile transformation, then drop anything that is still an outlier
    print('trait-wise outlier removal')
    quantile_data = pheno_quantile_transform(perturb_results[pheno_cols], pheno_cols)
    
    outlier_count = {}
    for pheno in pheno_cols:
        #outliers = ~filter_outliers_mad(perturb_results, pheno, outlier_cutoff=4, return_stats=False)
        outliers = ~filter_outliers_mad(quantile_data, pheno + '-q', outlier_cutoff=4, return_stats=False)
        if outliers.sum() < len(outliers) * 0.1:
            outlier_count[pheno] = int(outliers.sum())
            perturb_results.loc[outliers, pheno] = np.NaN
    
    with open(output_folder + 'outliers.json', 'w') as fp:
        json.dump(outlier_count, fp)
    #     pheno_data_filtered = perturb_results.copy(deep=True)
    #     quantile_data = pheno_quantile_transform(pheno_data_filtered[pheno_cols], pheno_cols)
    #     pheno_data_filtered = pheno_data_filtered.merge(quantile_data, left_index=True, right_index=True)
    #     pheno_columns_filtered = sorted(pheno_cols + list(quantile_data.columns))
    #     plot_pheno(pheno_data_filtered, pheno_columns_filtered, folder + 'pheno_filtered.pdf')
    
    # drop covariates that are not used in this run
    perturb_results = perturb_results.drop(['BMI', 'Tobacco', 'Glucocorticoid', 'Transplant'], axis=1)

    # only run gwas with at least 100 subjects
    if perturb_results.shape[0] > 100:
        name = perturb_
        # use the median response if multiple available
        perturb_results = perturb_results.groupby('PatientID').median().reset_index()            
        # prepare_phenotype_data assumes the index is the patient id
        perturb_results = perturb_results.set_index('PatientID')
        
        # write out plink input files
        prepare_phenotype_data(
            pheno_data=perturb_results,
            gwas_path=gwas_path,
            out_path=output_folder,
            id_type='PatientID',
            covariate_cols=['CHIP', 'BATCH', 'Sex', 'Age', 'Race', 'analysis_month', 'draw_analysis_hours'] + ['PC' + str(i+1) for i in range(10)],
            bedfile='imputed_merged',
            quantile_transform=True,
        )

        #perturb_results.to_parquet(
        #    f'{output_folder}/pheno_projected_filtered.parquet')
        perturb_results.to_csv(
            f'{output_folder}/gwasinput.txt', sep='\t'
        )

### single trait file

In [ ]:
for channel in tqdm(['wdf', 'ret', 'wnr', 'plt-f']):
    pheno_df_projected = pd.read_parquet(f'{analysis_path}/{channel}_pheno_projected.parquet')
    pheno_df_projected_filtered = pheno_df_projected.copy(deep=True)
    pheno_cols = list(set(pheno_df_projected.columns).difference(
        [
            'FIL', 'DATE', 'ETIM',
            'sample_id', 'patient_sample_id', 'standard_study_id',
            'PatientID', 'instrument', 'non_error',
            'perturbation', 'perturbation_num', 'sample', 'id_str', 'analysis_time', 'outlier',
            'analysis_date', 'analysis_month', 'Perturbation',
            'SpecimenTakenTimeDTS', 'rc_consent_datetime', 'draw_analysis_timedelta',
            'draw_analysis_hours', 'sysmex_datetime', 'Sex', 'Tobacco'
        ] + list(covariates_df.columns)
    ))

    # understand the data distributions better
    for perturb_ in tqdm(sorted(list(pheno_df_projected.Perturbation.unique()))):
        print(f'\n\n\n#### {perturb_} ####')
        perturb_results = pheno_df_projected.copy(deep=True).loc[pheno_df_projected.Perturbation == perturb_]

        # now drop outliers for each individual trait and condition
        # first do quantile transformation, then drop anything that is still an outlier
        print('trait-wise outlier removal')
        quantile_data = pheno_quantile_transform(perturb_results[pheno_cols], pheno_cols)

        outlier_count = {}
        for pheno in pheno_cols:
            outliers = ~filter_outliers_mad(quantile_data, pheno + '-q', outlier_cutoff=4, return_stats=False)
            if outliers.sum() < len(outliers) * 0.1:
                outlier_count[pheno] = int(outliers.sum())
                perturb_results.loc[outliers, pheno] = np.NaN
                # also drop them in the multi-perturbation dataset of blood readouts
                pheno_df_projected_filtered.loc[perturb_results.loc[outliers].index, pheno] = np.NaN

    pheno_df_projected_filtered.to_parquet(f'{analysis_path}/{channel}_pheno_projected_filtered.parquet')

In [ ]:
# make a single file of all sysmex traits
wdf_traits = pd.read_parquet(f'{analysis_path}/wdf_pheno_projected_filtered.parquet')
ret_traits = pd.read_parquet(f'{analysis_path}/ret_pheno_projected_filtered.parquet')
wnr_traits = pd.read_parquet(f'{analysis_path}/wnr_pheno_projected_filtered.parquet')
pltf_traits = pd.read_parquet(f'{analysis_path}/plt-f_pheno_projected_filtered.parquet')

In [ ]:
id_columns = [
    'sample', 'patient_sample_id', 'perturbation_num', 'standard_study_id',
    'PatientID', 'Age' , 'Sex', 'Race', 'BMI', 'Tobacco', 'Glucocorticoid', 'Transplant', 'Perturbation',
    'SpecimenTakenTimeDTS', 'rc_consent_datetime', 'draw_analysis_hours', 'sysmex_datetime',
    'analysis_time', 'analysis_month', 'analysis_date', 'instrument', 'ETIM', 'DATE'
             ]

wdf_traits = wdf_traits[id_columns + [c for c in wdf_traits.columns if not c in id_columns]]
wnr_traits = wnr_traits[id_columns + [c for c in wnr_traits.columns if not c in id_columns]]
ret_traits = ret_traits[id_columns + [c for c in ret_traits.columns if not c in id_columns]]
pltf_traits = pltf_traits[id_columns + [c for c in pltf_traits.columns if not c in id_columns]]

wdf_traits.columns = id_columns + ['wdf_' + c for c in wdf_traits.columns if not c in id_columns]
wnr_traits.columns = id_columns + ['wnr_' + c for c in wnr_traits.columns if not c in id_columns]
pltf_traits.columns = id_columns + ['pltf_' + c for c in pltf_traits.columns if not c in id_columns]
ret_traits.columns = id_columns + ['ret_' + c for c in ret_traits.columns if not c in id_columns]

all_traits = \
wdf_traits.merge(
    wnr_traits,
    left_on=id_columns,
    right_on=id_columns,
    how='outer'
)

all_traits = \
all_traits.merge(
    pltf_traits,
    left_on=id_columns,
    right_on=id_columns,
    how='outer'
)

all_traits = \
all_traits.merge(
    ret_traits,
    left_on=id_columns,
    right_on=id_columns,
    how='outer'
)

In [ ]:
all_traits.to_parquet(f'{analysis_path}/all_pheno_projected_filtered.parquet')

In [ ]:
# also calculate the median perturbation response per subject
all_traits_median = all_traits.groupby(['Perturbation', 'PatientID']).median().reset_index()
all_traits_median.to_parquet(f'{analysis_path}/all_pheno_projected_filtered_median.parquet')

In [ ]:
all_traits_median.to_csv(f'{analysis_path}/all_pheno_projected_filtered_median.tsv', sep='\t', index=False)

In [ ]:
len(all_traits_median.PatientID.unique())